# Business Problem

Chinese cuisine (Chinese dishes) originated from different regions of China and has become widespread in many other parts of the world. Regional cultural differences vary greatly amongst the different regions of China, giving rise to the different styles of food. Chinese cookery Introduction of Chinese Food has produced and matured in excess of the centuries, forming a wealthy cultural information it truly is characterized by fine collection of components, precise processing, certain care to your level of fire, and considerable nourishment. Confucius when mentioned: "Eating could be the utmost important point in daily life."

With increasing amount of Chinese population in Toronto, many chef have been looking into opportunities of opening new Chinese Restaurant. The challenge, besides offering a menu that powers the retention of customer, remains picking the most suitable location.

# Data Description

To begin with, the postal code information of the the neighbourhoods is required. We can grab them from here: https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

To complete the analysis, we will leverage Foursquare API as our data provider. It will be a sufficient source of creating dataframe and data visualization to determine the competition and location. Using the API, we gather information about the venues in each neighbourhood within a chosen radius of 100 meters. The data we retrieve will contain information on the venues within a specified distance of the longitude and latitude of the postcodes. The information we will obtain are as follows: Neighbourhood Neighbourhood Latitude and Longitude Venue: Name, Latitude, Longitude, Category.

We will use Pandas library for querying data and managing the dataframe. The result will be in JSON format. In addition to that, we will use Matplotlib for data visualization.



In [1]:
!conda update -n base -c defaults conda --yes
!conda install -c conda-forge folium --yes
!conda install -c conda lxml --yes
!pip install et_xmlfile --yes
!pip install bs4 --yes
!pip install html5lib --yes
!pip install lxml --yes
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize 
import matplotlib.pylab as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium
from geopy.geocoders import Nominatim
import bs4.builder._lxml 
from lxml import etree
import requests 
from bs4 import BeautifulSoup

Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - conda


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-3.2.0               |           py37_0         750 KB
    certifi-2019.11.28         |           py37_0         153 KB
    ------------------------------------------------------------
                                           Total:         903 KB

The following packages will be REMOVED:

  python_abi-3.7-1_cp37m

The following packages will be SUPERSEDED by a higher-priority channel:

  altair              conda-forge/noarch::altair-4.1.0-py_1 --> pkgs/main/osx-64::altair-3.2.0-py37_0
  certifi                                       conda-forge --> pkgs/main
  conda              conda-forge::conda-4.8.3-py37hc8dfbb8~ --> pkgs/main::conda-4.8.3-py37_0



certifi-2019.11.28   | 153 KB    | #####################

## Get neighborhood information from Wikipedia

In [2]:
req = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(req.content,features="lxml")
table = soup.find_all("table")[0]
a = pd.read_html(str(table))
df1 = pd.DataFrame(a[0])
df1['Borough'].replace('Not assigned', np.nan, inplace=True)
df1.dropna(subset=['Borough'], inplace=True)
n = df1['Neighborhood'].tolist()
nnew = []
for i in n:
    i = i.replace(" /" , ",")
    nnew.append(i)
df1.Neighborhood = nnew
df1.reset_index(inplace = True) 
del df1['index']
df1.head()

,Postal code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


## Find postal code locations on the map

In [3]:
df2 = pd.read_csv('http://cocl.us/Geospatial_data')
df2.columns = ['Postal code', 'Latitude', 'Longitude']
df2.head()

,Postal code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [4]:
df = pd.merge(df1, df2, on=['Postal code'], how='inner')
df[0:15]

,Postal code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


In [5]:
from geopy.geocoders import Nominatim

In [6]:
address = 'Toronto'

geolocator = Nominatim(user_agent="my_application")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [7]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, Neighborhood, pc in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df['Postal code']):
    label = '{}: {}, {}'.format(pc, Neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
map_toronto

## Find the Chinese Restaurants

In [1]:
# Foursquare 
CLIENT_ID = 'N0OU1GCXO2EA2CZBYTRCYVJZQDHDLXOLUUTZ4JZUJHSNDADD'
CLIENT_SECRET = 'N2GQTUVHNLE1ZRVA4SPHO2XZKRIALAGQN1Q4RWKS0HPPBDWI'
VERSION = '20200604'
search_query = "Chinese"
radius = 8000
LIMIT = 100

# city details
address = 'Toronto'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print("Toronto:\t",latitude, longitude)

NameError: name 'Nominatim' is not defined

In [9]:
url= 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
results = requests.get(url).json()
venues = results['response']['venues']
# making a dataframe
dfv = json_normalize(venues)
dfv

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  """


,id,name,categories,referralId,hasPerk,location.address,location.lat,location.lng,location.labeledLatLngs,location.distance,location.postalCode,location.cc,location.city,location.state,location.country,location.formattedAddress,location.crossStreet
0,5455c935498eb34a38436958,Marhaba Lebanese Cuisine,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1588361673,False,"236 Sherbourne Street, Unit #5",43.658378,-79.370972,"[{'label': 'display', 'lat': 43.65837775193231...",1177,M5A 2R7,CA,Toronto,ON,Canada,"[236 Sherbourne Street, Unit #5, Toronto ON M5...",NaN
1,4bae8f9ef964a520d0c03be3,Healthy Lebanese Cuisine & Juice Bar,"[{'id': '4bf58dd8d48988d10b941735', 'name': 'F...",v-1588361673,False,NaN,43.665504,-79.384876,"[{'label': 'display', 'lat': 43.665504, 'lng':...",1340,NaN,CA,NaN,NaN,Canada,[Canada],NaN
2,4ba3a9bcf964a520815138e3,Wrap & Grab Lebanese Cuisine,"[{'id': '4bf58dd8d48988d115941735', 'name': 'M...",v-1588361673,False,618 Yonge St,43.666040,-79.385190,"[{'label': 'display', 'lat': 43.6660400448863,...",1401,NaN,CA,Toronto,ON,Canada,"[618 Yonge St, Toronto ON, Canada]",NaN
3,5d8653a66d03900008abeac2,Beiteddine Lebanese Restaurant,"[{'id': '58daa1558bbb0b01f18ec1cd', 'name': 'L...",v-1588361673,False,NaN,43.678065,-79.348542,"[{'label': 'display', 'lat': 43.678065, 'lng':...",3951,M4K 1P5,CA,Toronto,ON,Canada,"[Toronto ON M4K 1P5, Canada]",NaN
4,4db03f075da32cf2df4546bf,Honorary Consulate of Lebanon,"[{'id': '4bf58dd8d48988d12c951735', 'name': 'E...",v-1588361673,False,2224 Dundas St W,43.654147,-79.451695,"[{'label': 'display', 'lat': 43.6541469637391,...",5458,M6R 1X3,CA,Toronto,ON,Canada,"[2224 Dundas St W (at Roncesvalles Av), Toront...",at Roncesvalles Av


In [10]:
# Now for some data-wrangling. Drop the columns I don't need. 
filtered_columns = ['name', 'categories'] + [col for col in dfv.columns if col.startswith('location.')] +  ['id']
dfv_filtered = dfv.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dfv_filtered['categories'] = dfv_filtered.apply(get_category_type, axis=1)
# clean column names by keeping only last term
dfv_filtered.columns = [column.split('.')[-1] for column in dfv_filtered.columns]

dfv_filtered

,name,categories,address,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,crossStreet,id
0,Marhaba Lebanese Cuisine,Middle Eastern Restaurant,"236 Sherbourne Street, Unit #5",43.658378,-79.370972,"[{'label': 'display', 'lat': 43.65837775193231...",1177,M5A 2R7,CA,Toronto,ON,Canada,"[236 Sherbourne Street, Unit #5, Toronto ON M5...",NaN,5455c935498eb34a38436958
1,Healthy Lebanese Cuisine & Juice Bar,Falafel Restaurant,NaN,43.665504,-79.384876,"[{'label': 'display', 'lat': 43.665504, 'lng':...",1340,NaN,CA,NaN,NaN,Canada,[Canada],NaN,4bae8f9ef964a520d0c03be3
2,Wrap & Grab Lebanese Cuisine,Middle Eastern Restaurant,618 Yonge St,43.666040,-79.385190,"[{'label': 'display', 'lat': 43.6660400448863,...",1401,NaN,CA,Toronto,ON,Canada,"[618 Yonge St, Toronto ON, Canada]",NaN,4ba3a9bcf964a520815138e3
3,Beiteddine Lebanese Restaurant,Lebanese Restaurant,NaN,43.678065,-79.348542,"[{'label': 'display', 'lat': 43.678065, 'lng':...",3951,M4K 1P5,CA,Toronto,ON,Canada,"[Toronto ON M4K 1P5, Canada]",NaN,5d8653a66d03900008abeac2
4,Honorary Consulate of Lebanon,Embassy / Consulate,2224 Dundas St W,43.654147,-79.451695,"[{'label': 'display', 'lat': 43.6541469637391,...",5458,M6R 1X3,CA,Toronto,ON,Canada,"[2224 Dundas St W (at Roncesvalles Av), Toront...",at Roncesvalles Av,4db03f075da32cf2df4546bf


In [12]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the City Hall

# add a red  marker to represent the City Hall
folium.Circle(
    [latitude, longitude],
    radius=20,
    color='red',
    popup='City Hall',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the Coffee Shops as blue markers
for lat, lng, label in zip(dfv_filtered.lat, dfv_filtered.lng, dfv_filtered.categories):
    folium.Circle(
        [lat, lng],
        radius=30,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Show Chinese Restaurants according to postal code

In [13]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, borough, Neighborhood, pc in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighborhood'], df['Postal code']):
    label = '{}: {}, {}'.format(pc, Neighborhood, borough)
    label = folium.Popup(label)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_toronto)  
    
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13) # generate map centred around the City Hall

# add the Coffee Shops as blue markers
for lat, lng, label in zip(dfv_filtered.lat, dfv_filtered.lng, dfv_filtered.categories):
    folium.Circle(
        [lat, lng],
        radius=15,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(map_toronto)

# display map
map_toronto